**Data Inspection, Cleaning and Preparation**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Import the necessary modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import math
import numpy as np
import os

In [ ]:
def df_check(df_to_check):
    """Function to generate information for a data frame"""
    # Generate a summary for NaN, zero and unique values
    nan_count = df_to_check.isna().sum()
    unique_count = df_to_check.nunique()
    zero_count = (df_to_check == 0).sum()
    data_type = df_to_check.dtypes

    # Merge the summaries into one dataframe for inspection
    df_check = pd.concat([nan_count,unique_count, zero_count, data_type],axis =1)
    df_check.columns = ['NaN Count','Unique Count','Zero Count', 'Data Type']
    print(df_check)


In [ ]:
# Function to load a CSV file into a DataFrame
def load_csv(file_path, parse_dates=None, dtype=None):
    """Load a CSV file into a DataFrame."""
    return pd.read_csv(file_path, sep=',', header=0, parse_dates=parse_dates, dtype=dtype)


In [ ]:
# Function to preprocess demand data (convert to datetime and aggregate)
def preprocess_demand_data(df, date_format='%d/%m/%Y %H:%M', freq='D'):
    """Convert DATETIME to datetime format and aggregate the demand data by frequency."""
    df['DATETIME'] = pd.to_datetime(df['DATETIME'], format=date_format)
    df.set_index('DATETIME', inplace=True)
    df.drop('REGIONID', axis=1, inplace=True)
    return df.resample(freq).sum()

In [ ]:
# Function to preprocess temperature data (convert to datetime and calculate mean)
def preprocess_temperature_data(df, date_format='%d/%m/%Y %H:%M', freq='D'):
    """Convert DATETIME to datetime format and calculate mean temperature by frequency."""
    df['DATETIME'] = pd.to_datetime(df['DATETIME'], format=date_format)
    df.set_index('DATETIME', inplace=True)
    df.drop('LOCATION', axis=1, inplace=True)
    return df.resample(freq).mean()

In [ ]:
# Function to preprocess SILO data
def preprocess_silo_data(df, freq='D'):
    """Select relevant columns, calculate RH, and aggregate SILO data by frequency."""
    df['Date2'] = pd.to_datetime(df['Date2'], format='%d/%m/%Y')
    df['RH'] = (df['RHmaxT'] + df['RHminT']) / 2  # Calculate RH as the average of RHMaxT and RHminT
    df_filtered = df[['Date2', 'Rain', 'Evap', 'Radn', 'VP', 'RH']]  # Select relevant columns
    df_filtered.set_index('Date2', inplace=True)

    # Aggregating data based on frequency
    if freq == 'D':
        return df_filtered.resample(freq).mean()
    else:
        agg_dict = {
            'Rain': 'sum',
            'Evap': 'sum',
            'Radn': 'sum',
            'VP': 'sum',
            'RH': 'mean'
        }
        return df_filtered.resample(freq).agg(agg_dict)

In [ ]:
# Function to perform all preprocessing steps
def preprocess_all_data(demand_file, temp_file, silo_file, demand_date_format, temp_date_format, freq='D'):
    """Load, preprocess demand, temperature, and SILO data."""
    # Load the data
    demand_df = load_csv(demand_file, parse_dates=['DATETIME'], dtype={'TOTALDEMAND': float, 'REGIONID': str})
    temp_df = load_csv(temp_file, parse_dates=['DATETIME'])
    silo_df = load_csv(silo_file, parse_dates=['Date2'])

    # Preprocess the data
    demand_processed_df = preprocess_demand_data(demand_df, date_format=demand_date_format, freq=freq)
    temp_processed_df = preprocess_temperature_data(temp_df, date_format=temp_date_format, freq=freq)
    silo_processed_df = preprocess_silo_data(silo_df, freq=freq)

    # Data Check
    print(f"Data check for {demand_file.split('/')[-1]}:")
    df_check(demand_processed_df)
    print("\n")
    print(f"Data check for {temp_file.split('/')[-1]}:")
    df_check(temp_processed_df)
    print("\n")
    print(f"Data check for {silo_file.split('/')[-1]}:")
    df_check(silo_processed_df)

    return demand_processed_df, temp_processed_df, silo_processed_df

In [ ]:
# Set the working directory
os.chdir('/content/drive/MyDrive/data/Group10_project')

# File paths
nsw_demand_file = 'data/Raw Files/totaldemand_nsw.csv'
qld_demand_file = 'data/Raw Files/totaldemand_qld.csv'
nsw_temp_file = 'data/Raw Files/temperature_nsw.csv'
qld_temp_file = 'data/Raw Files/temprature_qld.csv'
nsw_silo_file = 'data/Raw Files/66137 BANKSTOWN.csv'
qld_silo_file = 'data/Raw Files/40842 BRISBANE.csv'

In [ ]:
# Preprocess NSW data
daily_totaldemand_nsw, daily_temperature_nsw, daily_silo_nsw = preprocess_all_data(
    nsw_demand_file, nsw_temp_file, nsw_silo_file, '%d/%m/%Y %H:%M', '%d/%m/%Y %H:%M', freq='D'
)

# Preprocess QLD data
daily_totaldemand_qld, daily_temperature_qld, daily_silo_qld = preprocess_all_data(
    qld_demand_file, qld_temp_file, qld_silo_file, '%d/%m/%Y %H:%M', '%d/%m/%Y %H:%M', freq='D'
)


Data check for totaldemand_nsw.csv:
             NaN Count  Unique Count  Zero Count Data Type
TOTALDEMAND          0          4095           0   float64


Data check for temperature_nsw.csv:
             NaN Count  Unique Count  Zero Count Data Type
TEMPERATURE          3          3938           0   float64


Data check for 66137 BANKSTOWN.csv:
      NaN Count  Unique Count  Zero Count Data Type
Rain          0           234        3592   float64
Evap          0           129           1   float64
Radn          0           327           0   float64
VP            0           204           0   float64
RH            0          1287           0   float64
Data check for totaldemand_qld.csv:
             NaN Count  Unique Count  Zero Count Data Type
TOTALDEMAND          0          4094           0   float64


Data check for temprature_qld.csv:
             NaN Count  Unique Count  Zero Count Data Type
TEMPERATURE          7          3767           0   float64


Data check for 40842 BRISBANE

In [ ]:
# Function to generate aggregates based on the dataframe columns and frequency
def generate_aggregates(df, freq='W-MON'):
    """Generate aggregates based on frequency and available columns in the dataframe."""
    # Define the aggregation rules based on available columns in the dataframe
    agg_dict = {}

    # Check and add aggregation rules based on column presence
    if 'Rain' in df.columns:
        agg_dict['Rain'] = 'sum'
    if 'Evap' in df.columns:
        agg_dict['Evap'] = 'sum'
    if 'Radn' in df.columns:
        agg_dict['Radn'] = 'sum'
    if 'VP' in df.columns:
        agg_dict['VP'] = 'sum'
    if 'RH' in df.columns:
        agg_dict['RH'] = 'mean'
    if 'TOTALDEMAND' in df.columns:
        agg_dict['TOTALDEMAND'] = 'sum'
    if 'TEMPERATURE' in df.columns:
        agg_dict['TEMPERATURE'] = 'mean'

    # If no specific aggregation rules are needed, just use sum for numeric columns
    if not agg_dict:
        return df.resample(freq).sum()

    # Aggregate based on the defined aggregation rules
    return df.resample(freq).agg(agg_dict)

# Weekly aggregates
weekly_totaldemand_nsw = generate_aggregates(daily_totaldemand_nsw, freq='W-MON')
weekly_totaldemand_qld = generate_aggregates(daily_totaldemand_qld, freq='W-MON')
weekly_temperature_nsw = generate_aggregates(daily_temperature_nsw, freq='W-MON')
weekly_temperature_qld = generate_aggregates(daily_temperature_qld, freq='W-MON')
weekly_silo_nsw = generate_aggregates(daily_silo_nsw, freq='W-MON')
weekly_silo_qld = generate_aggregates(daily_silo_qld, freq='W-MON')

# Monthly aggregates
monthly_totaldemand_nsw = generate_aggregates(daily_totaldemand_nsw, freq='M')
monthly_totaldemand_qld = generate_aggregates(daily_totaldemand_qld, freq='M')
monthly_temperature_nsw = generate_aggregates(daily_temperature_nsw, freq='M')
monthly_temperature_qld = generate_aggregates(daily_temperature_qld, freq='M')
monthly_silo_nsw = generate_aggregates(daily_silo_nsw, freq='M')
monthly_silo_qld = generate_aggregates(daily_silo_qld, freq='M')

In [ ]:
# Function to merge demand, temperature, and SILO data
def merge_dataframes(demand_df, temp_df, silo_df, join_type='left'):
    """
    Merge demand, temperature, and SILO dataframes using the specified join type.
    Arguments:
    - demand_df: DataFrame containing demand data.
    - temp_df: DataFrame containing temperature data.
    - silo_df: DataFrame containing SILO data.
    - join_type: Type of join to be used for merging (default is 'left').
    Returns:
    - merged_df: DataFrame with merged data.
    """
    # Merge demand and temperature data
    merged_df = demand_df.merge(temp_df, how=join_type, left_index=True, right_index=True)
    # Merge the result with SILO data
    merged_df = merged_df.merge(silo_df, how=join_type, left_index=True, right_index=True)
    return merged_df

# Merge daily data
daily_merged_nsw = merge_dataframes(daily_totaldemand_nsw, daily_temperature_nsw, daily_silo_nsw)
daily_merged_qld = merge_dataframes(daily_totaldemand_qld, daily_temperature_qld, daily_silo_qld)

# Merge weekly data
weekly_merged_nsw = merge_dataframes(weekly_totaldemand_nsw, weekly_temperature_nsw, weekly_silo_nsw)
weekly_merged_qld = merge_dataframes(weekly_totaldemand_qld, weekly_temperature_qld, weekly_silo_qld)

# Merge monthly data
monthly_merged_nsw = merge_dataframes(monthly_totaldemand_nsw, monthly_temperature_nsw, monthly_silo_nsw)
monthly_merged_qld = merge_dataframes(monthly_totaldemand_qld, monthly_temperature_qld, monthly_silo_qld)

# Display the merged data (optional)
print("Daily Merged NSW Data:")
print(daily_merged_nsw.head())

print("\nWeekly Merged NSW Data:")
print(weekly_merged_nsw.head())

print("\nMonthly Merged NSW Data:")
print(monthly_merged_nsw.head())

Daily Merged NSW Data:
            TOTALDEMAND  TEMPERATURE  Rain  Evap  Radn    VP     RH
DATETIME                                                           
2010-01-01    374464.94    25.094000   0.2   5.2  14.6  23.1  76.35
2010-01-02    384838.07    24.765385   0.0   5.9  18.4  22.0  67.80
2010-01-03    355195.18    19.429825  15.2   1.9   7.3  19.1  83.25
2010-01-04    395602.57    20.625926   0.2   3.4  14.4  18.2  75.15
2010-01-05    423735.03    22.660417   0.0   6.8  28.1  16.9  65.35

Weekly Merged NSW Data:
            TOTALDEMAND  TEMPERATURE  Rain  Evap   Radn     VP         RH
DATETIME                                                                 
2010-01-04   1510100.76    22.478784  15.6  16.4   54.7   82.4  75.637500
2010-01-11   3037003.89    23.787509   0.4  46.8  184.1  130.2  63.350000
2010-01-18   3029758.51    22.719923   8.4  44.6  156.7  132.6  65.550000
2010-01-25   3127293.47    23.591350   3.2  51.9  177.0  112.4  55.835714
2010-02-01   3078396.21    23.98